In [96]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from tqdm import tqdm
import os
from datetime import datetime

In [97]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [98]:
class Estimatio_LSTM(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,seq_len,num_layers):
        super().__init__()
        self.seq_len = seq_len
        self.num_layers =num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.lstm = nn.LSTM(input_size, hidden_size,num_layers=num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size)

    
    def forward(self,x):
        # x [batch ,times_seires,input_size]
        out, _ = self.lstm(x)
        # out [batch,hidden_szie]
        out = self.fc(out[:,-1])
        return out
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.num_layers, self.seq_len, self.hidden_size),
            torch.zeros(self.num_layers, self.seq_len, self.hidden_size)
        )

In [100]:
class Estimation_My():
    def __init__(self):

        if torch.cuda.is_available():
            self.device = torch.device('cuda:0')
            print('Cuda is available')
        else:
            print('there is no Cuda')
            self.device = torch.device('cpu')

        torch.cuda.set_device(self.device)

    def normalization(self,data):
            mean = torch.mean(data, dim=0)
            std = torch.std(data, dim=0)
            result = 100 * (data-mean) / std
            
            return result
    
    def make_dir(self,log_dir,pt_dir):
        self.pt_dir = os.path.join(pt_dir,str(self.hidden_size))
        self.log_dir = log_dir
        if os.path.isdir(self.log_dir) == False:
            os.makedirs(self.log_dir)
        if os.path.isdir(self.pt_dir) == False:
            os.makedirs(self.pt_dir)

    def data_loader(self,Data_path,batch_size =32, seq_len=50,split = 0.2):
        self.seq_len = seq_len
        Data = pd.read_csv(Data_path)
        y = Data.pop(str(Data.columns[-1])).values


        Data = torch.tensor(Data.values, dtype=torch.float32)
        Data = self.normalization(Data)

        y = torch.tensor(y,dtype=torch.float32)
        y = self.normalization(y)
        data_X = []
        data_Y = []
        for i in range(0, len(Data)-seq_len):
            if i+seq_len > len(Data):
                break
            _x = Data[i:i+seq_len, :]
            _y = y[i:i+seq_len]

            data_X.append(_x)  
            data_Y.append(_y)
        print(len(data_Y))
        data_X = torch.FloatTensor(np.array(data_X))
        data_Y = torch.FloatTensor(np.array(data_Y))

        
        data_X = data_X.to(self.device)
        data_Y = data_Y.to(self.device)

        X_train, X_val = random_split(data_X,[int(len(data_X)*split),len(data_X)-int(len(data_X)*split)])
        y_train, y_val = random_split(data_Y,[int(len(data_X)*split),len(data_X)-int(len(data_X)*split)])
        train_dataset = CustomDataset(X_train,y_train)
        val_dataset = CustomDataset(X_val,y_val)

        self.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        self.val_loader = DataLoader(val_dataset, batch_size=batch_size)       

    def check_the_dataset(self):
        print(self.train_loader)
        print(next(iter(self.train_loader)))
    
    
    def model_setting(self,hidden_size,output_size=1,num_layers=1):
        self.hidden_size = hidden_size
        sample_batch = next(iter(self.train_loader))
        self.input_size = sample_batch[0].shape[2]
        print(self.input_size)
        self.model = Estimatio_LSTM(self.input_size,self.hidden_size,output_size,self.seq_len, num_layers)
        self.model.to(self.device)

    def train_setting(self,lr=1e-4, loss = nn.L1Loss()):
        self.criterion = loss
        self.lr = lr
        self.optimizer = optim.Adam(self.model.parameters(),lr=lr)
        
    
    def save_checkpoint(self, filename):
        torch.save(self.model, filename)

    
    def train(self,epoch):
        self.loss_log=[]
        for epoch in tqdm(range(epoch)):
            total_train_loss = 0
            self.model.train()
            for inputs, labels in self.train_loader:
                self.model.reset_hidden_state()
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                outputs = outputs.reshape((-1,1))
                train_loss = self.criterion(outputs, labels)
                train_loss.backward()
                self.optimizer.step()
                total_train_loss += train_loss.item()
            avg_train_loss =total_train_loss/len(self.train_loader)

            # Validation loop
            self.model.eval()
            with torch.no_grad():
                val_loss =0
                for inputs, labels in self.val_loader:
                    outputs = self.model(inputs)
                    val_loss += self.criterion(outputs, labels).item()
                avg_val_loss = val_loss/ len(self.val_loader)
            
            self.loss_log.append([avg_train_loss,avg_val_loss])

            print(f'Epoch {epoch+1}, Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')
            self.save_checkpoint(os.path.join(self.pt_dir,f'model_epoch_{epoch}.pt'))

In [101]:
batch_size = 2
lr = 1e-4
hidden_size = 5
depth = 5
epoch = 500
#path
today = datetime.today()
Data_path = os.path.join('../Data/','TV_JM_145120.csv')
log_dir = os.path.join('../Data/ML',str(today.date()),'log') 
pt_dir = os.path.join('../Data/ML',str(today.date()),"pt")

model =  Estimation_My()
model.data_loader(Data_path,batch_size=batch_size)
model.model_setting(hidden_size=10)
model.train_setting()
model.make_dir(pt_dir=pt_dir,log_dir=log_dir)
model.train(epoch=100)

Cuda is available
2983
11


  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\jm538\anaconda3\envs\TV\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([2, 50])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
c:\Users\jm538\anaconda3\envs\TV\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([1, 50])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
  1%|          | 1/100 [00:00<01:37,  1.02it/s]

Epoch 1, Loss: 78.6460, Validation Loss: 80.2296


  2%|▏         | 2/100 [00:01<01:28,  1.11it/s]

Epoch 2, Loss: 78.6389, Validation Loss: 80.2285


...

Epoch 98, Loss: 78.3025, Validation Loss: 80.1739


 99%|█████████▉| 99/100 [01:22<00:00,  1.20it/s]

Epoch 99, Loss: 78.3005, Validation Loss: 80.1726


100%|██████████| 100/100 [01:22<00:00,  1.21it/s]

Epoch 100, Loss: 78.2964, Validation Loss: 80.1717
